In this notebook, we'll be performing exploratory text analysis on the notes from Durham's CFS data.  To look at word frequencies, we have to combine all the words into a single corpus.

In [ ]:
import nltk
import dataset

db = dataset.connect("postgresql://jnance:@localhost:5432/cfs")

In [2]:
notes = []

for row in db.query("SELECT body FROM note;"):
    if row['body']:
        notes += row['body'].lower().split()
    
txt = nltk.Text(notes)
fd = nltk.FreqDist(txt)

/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))
/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'extent'
  (attype, name))


Here we can see the most frequent words in the corpus.  Many of these will probably become domain stopwords. (note: must download the nltk stopwords corpus using `nltk.download()` before this will work)

In [12]:
print("========= TOP 100 WORDS =========")
l = fd.most_common()
ndx = 0
num = 1
ignore = nltk.corpus.stopwords.words('english')
while (num <= 100 and ndx < len(l)):
    if l[ndx][0] not in ignore:
        print(str(num) + ". " + l[ndx][0])
        num += 1
    ndx += 1
print("\n")

========= TOP 100 WORDS =========
1. caller
2. [epd]
3. -
4. questions:
5. ***
6. scene.
7. is:
8. incident
9. vehicle
10. response:
11. chief
12. complaint:
13. statement:
14. 1.
15. 2.
16. 3.
17. 4.
18. 5.
19. 6.
20. info
21. /
22. 7.
23. one
24. code:
25. person
26. .
27. description
28. known
29. involved.
30. involves
31. dispatch
32. male
33. 8.
34. suspect
35. call
36. 9.
37. weapons
38. adv
39. 1.suspect:
40. back
41. 10.
42. location
43. race:
44. priority
45. gender:
46. 11.
47. color:
48. age:
49. danger.
50. clothing:
51. alarm
52. party
53. cb
54. happened
55. blk
56. business/resident/owner
57. suspect`s
58. reported
59. known.
60. 12.
61. phone
62. number
63. traffic
64. left
65. suspect/person
66. 2
67. progress.
68. area.
69. involved
70. female
71. victim
72. calling
73. suspicious
74. black
75. pd
76. name
77. responsible
78. n/a
79. make:
80. medical
81. mentioned.
82. area
83. needs
84. door
85. 13.
86. s/he
87. body:
88. [fire]
89. 2nd
90. victim.
91. disturbance


Many of these words are part of the call scripts transcribed by the CAD system, describing the caller's answers to specific questions asked by the dispatcher.  We'll probably need to add the script words to a stopword list, but we can keep the descriptive words that go along with them.

In [14]:
dpd_stopwords = set(nltk.corpus.stopwords.words('english'))

domain_stopwords = set(['caller', '[epd]', '-', 'questions:', 'scene.', '***', 'is:', 'incident', 'response:', 'chief',
                       'complaint:', 'statement:', 'info', 'description', 'known', 'involved', 'involves', 'dispatch',
                       '1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.', '11.', '12.', '13.', '/', 'code:',
                       '.', 'call', 'suspect', 'adv', '1.suspect:', 'location', 'race:', 'priority', 'gender:',
                       'color:', 'age:', 'clothing:', 'cb', 'suspect`s', 'reported', 'known.', 'suspect/person',
                       'progress.', 'area.', 'pd', 'name', 'n/a', 'make:', 'area', 's/he',
                       '[fire]', 'model:'])

dpd_stopwords.update(domain_stopwords)

Trying again with domain stopwords added:

In [16]:
print("========= TOP 100 WORDS =========")
l = fd.most_common()
ndx = 0
num = 1
ignore = dpd_stopwords
while (num <= 100 and ndx < len(l)):
    if l[ndx][0] not in ignore:
        print(str(num) + ". " + l[ndx][0])
        num += 1
    ndx += 1
print("\n")

========= TOP 100 WORDS =========
1. vehicle
2. one
3. person
4. involved.
5. male
6. weapons
7. back
8. danger.
9. alarm
10. party
11. happened
12. blk
13. business/resident/owner
14. phone
15. number
16. traffic
17. left
18. 2
19. female
20. victim
21. calling
22. suspicious
23. black
24. responsible
25. medical
26. mentioned.
27. needs
28. door
29. body:
30. 2nd
31. victim.
32. disturbance
33. see
34. ft
35. disturbance.
36. white
37. veh
38. line
39. open
40. theft
41. attention.
42. past:
43. aborted
44. leave
45. alarms
46. reportedly
47. caller.
48. 14.
49. drugs
50. monitoring
51. company.
52. two
53. activation
54. still
55. keyholder/owner
56. blocking
57. emergency
58. car
59. alarm.
60. law
61. incident.
62. contacted.
63. property
64. st
65. someone
66. front
67. going
68. advised
69. house
70. vm
71. suspect/person/vehicle
72. 1
73. 4
74. vehicles
75. blue
76. burglary/intrusion
77. tty
78. unk
79. no:
80. alcohol
81. 3
82. officer
83. vehicle.
84. people
85. 3rd
86. migh